In [4]:
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk
from IPython.display import display
from infrastructure.gpt4 import GPT4

sys.path.append('../src')


from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMapPlotly

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [(x, y) for x, y in geom.exterior.coords]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms][0]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)

comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=1000,
)



/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_10566/2974657862.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = comunas.geom.centroid.y,
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykern

In [6]:

from application.services.gpt4_map_selector import get_available_tools

gpt4 = GPT4()
prompt = "Generate a map that can answer the question 'How many votes did Emilia Rios obtained in 2021 municipal elections (alcaldes)?'"
tools = get_available_tools(comunas)


In [ ]:
comuna

In [9]:
gpt4.function_calling(prompt, system_prompt=None, tools=tools)

[ChatCompletionMessageToolCall(id='call_rTrFWG1jodpenY1n3uvZA4PU', function=Function(arguments='{"title": "Votes for Emilia Ríos in 2021 Municipal Elections (Alcaldes)", "value_column": "score"}', name='create_choropleth_map'), type='function')]

In [1]:
import inspect
def hola(a, b):
    pass
args = inspect.getfullargspec(hola).args
args_list = [str(arg) for arg in args]
print(args_list)

['a', 'b']


In [ ]:
select_map